In [40]:
import gensim
import fastText

import pandas as pd
import numpy as np

import pymorphy2
from tqdm import tqdm_notebook

from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer
from keras.utils.np_utils import to_categorical
from keras.layers import Conv1D, MaxPooling1D, Embedding, Reshape, LSTM, BatchNormalization
from keras.layers import Dense, Input, Flatten, Dropout, Merge, Activation, Concatenate
from keras.models import Model, Sequential
from keras.optimizers import Adadelta, RMSprop
from keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint

import re
import os
import csv
import string
import warnings
warnings.filterwarnings('ignore')

np.random.seed(42)

import tensorflow as tf
if tf.test.gpu_device_name():
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))
else:
    print("Please install GPU version of TF")

Default GPU Device: /device:GPU:0


## Preparing OpenSubtitles for w2v fitting

In [41]:
def process(source, total):
    processed = source + '.processed'

    with open(source, encoding='utf-8') as source_f, open(processed, 'w', encoding='utf-8') as processed_f:
        for line in tqdm_notebook(source_f, total=total):
            #print(line.lower().translate(table))
            processed_f.write(line.lower().translate(table))

#table = str.maketrans({key: None for key in string.punctuation})
#process('../w2v/os_dataset/OpenSubtitles2018.en-ru.ru', 25910105)

## Loading vectors

In [42]:
w2v = fastText.load_model('../w2v/os_model/os_model_2018.bin')

In [43]:
m = gensim.models.KeyedVectors.load('os_model_2018.gen.bin')

In [44]:
def similarity(v1, v2):
    return np.sum(v1 * v2) / (np.sqrt(np.sum(v1 ** 2)) * np.sqrt(np.sum(v2 ** 2)))

## Data parsing

In [45]:
def parsing_Xtrain(path):
    data_list = []
    with open(path, encoding='utf-8') as tsv:
        data_list = [line.split('\t') for line in tsv]
    data = pd.DataFrame(data_list, columns=['context_id','context_2',
                                            'context_1','context_0',
                                            'reply_id','reply','label',
                                            'confidence'],)
    return data

def parsing_Xtest(path):
    data_list = []
    with open(path, encoding='utf-8') as tsv:
        data_list = [line.split('\t') for line in tsv]
    data = pd.DataFrame(data_list, columns=['context_id','context_2',
                                            'context_1','context_0',
                                            'reply_id','reply'],
                       )
    return data

## Data cleaning

In [46]:
morph = pymorphy2.MorphAnalyzer()
def clear_text(text):        
    text = re.sub(r'[^\w-]+', ' ', text)
    text = re.sub(r'\s+', ' ', text)
    text = re.sub(r'\s+$', '', text)
    text = text.lower()
    text = text.replace('ё', 'е')
    return text

from functools import lru_cache
@lru_cache(maxsize=2**32)
def normalize_word(word):
    return morph.normal_forms(word)[0]

def normalize_text(text):
    return ' '.join([normalize_word(word) for word in text.split(' ')])

def clear_data(X, norm=False):
    X.context_2 = X.context_2.map(clear_text)
    X.context_1 = X.context_1.map(clear_text)
    X.context_0 = X.context_0.map(clear_text)
    X.reply = X.reply.map(clear_text)
    if norm == True:
        X.context_2 = X.context_2.map(normalize_text)
        X.context_1 = X.context_1.map(normalize_text)
        X.context_0 = X.context_0.map(normalize_text)
        X.reply = X.reply.map(normalize_text)
    return X

In [47]:
Xtrain = parsing_Xtrain('yac_data/train.tsv')
Xtest = parsing_Xtest('yac_data/public.tsv')

In [48]:
Xtrain.label = Xtrain.label.map({'good' : int(2),
                                 'bad' : int(0),
                                 'neutral' : int(1)})

In [49]:
%%time
Xtest = clear_data(Xtest, False)
Xtrain = clear_data(Xtrain, False)

y = Xtrain.label.values
y = y.astype(np.int)
label = to_categorical(y)
label.shape

def clear_conf(conf):
    return conf[:-1]
Xtrain.confidence = Xtrain.confidence.map(clear_conf)
Xtrain.confidence = Xtrain.confidence.astype(np.float32)

CPU times: user 3.48 s, sys: 20.8 ms, total: 3.5 s
Wall time: 3.46 s


## Trying TF-IDF

In [50]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [51]:
corpus = []
for sample in Xtrain.values:
    #print(sample)
    corpus.append(sample[2] + ' ' + sample[1] + ' ' + sample[3] + ' ' + sample[5])
vectorizer = TfidfVectorizer()
vectorizer.fit(corpus)
#vectorizer.idf_
#train = vectorizer.transform(corpus)
#a  = pd.DataFrame(corpus).to_csv('corus.txt', index=False)

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
        stop_words=None, strip_accents=None, sublinear_tf=False,
        token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
        vocabulary=None)

In [52]:
def get_idf_sentence(sent, vocab, idf):
    if len(sent) == 0:
        return 0
    sum_idf = 0
    for word in sent:
        if word in vocab:
            sum_idf += idf[vocab[word]]
        else:
            continue
    return sum_idf / len(sent)

## Get top words

In [53]:
def get_top_important_word(sent, vectorizer, n=5):
    vocab = vectorizer.vocabulary_
    idf = vectorizer.idf_
    v_sent = w2v.get_sentence_vector(' '.join(sent))
    imp_list = []
    for word in sent:
        #v_word = w2v.get_word_vector(word)
        #score = similarity(v_sent, v_word)
        if word in vocab:
            score = idf[vocab[word]]
            imp_list.append((score, word))
        else:
            continue
    imp_list.sort(reverse=True)
    if len(imp_list) >= n:
        th = imp_list[n - 1][0]
    else:
        th = 0
    imp_list = []
    for word in sent:
        if word in vocab:
            score = idf[vocab[word]]
            #print(score, th)
            if score >= th:
                imp_list.append(word)
        else:
            continue
    #print(imp_list[:5])
    return imp_list[:5]

def get_important_words(X, vectorizer, w2v):
    data = []
    print('Getting data...')
    for sample in tqdm_notebook(X.values):
        c2 = sample[1].split()
        c1 = sample[2].split()
        c0 = sample[3].split()
        r = sample[5].split()
        
        c2 = get_top_important_word(c2, vectorizer)
        c1 = get_top_important_word(c1, vectorizer)
        c0 = get_top_important_word(c0, vectorizer)
        r = get_top_important_word(r, vectorizer)
        
        c2_f = np.zeros((5, 200))
        c1_f = np.zeros((5, 200))
        c0_f = np.zeros((5, 200))
        r_f = np.zeros((5, 200))
        for i in range(len(c2)):
            c2_f[i] = w2v.get_word_vector(c2[i])
        for i in range(len(c1)):
            c1_f[i] = w2v.get_word_vector(c1[i])
        for i in range(len(c0)):
            c0_f[i] = w2v.get_word_vector(c0[i])
        for i in range(len(r)):
            r_f[i] = w2v.get_word_vector(r[i])
            
        data.append((c2_f, c1_f, c0_f, r_f))
        
    print('...Done')
    data = pd.DataFrame(data, columns=['c2', 'c1', 'c0', 'r'])

    c2 = np.zeros((len(data), 5, 200))
    print('Progress: 0%')
    for i in range(len(data)):
        c2[i] = data.c2[i]
    c1 = np.zeros((len(data), 5, 200))
    print('Progress: 25%')
    for i in range(len(data)):
        c1[i] = data.c1[i]
    c0 = np.zeros((len(data), 5, 200))
    print('Progress: 50%')
    for i in range(len(data)):
        c0[i] = data.c0[i]
    r = np.zeros((len(data), 5, 200))
    print('Progress: 75%')
    for i in range(len(data)):
        r[i] = data.r[i]
    print('Progress: 100%')
    return c2,c1,c0,r

In [54]:
#top_c2, top_c1 = np.load('yac_data/top_words/top_c2_full.npy'), np.load('yac_data/top_words/top_c1_full.npy')
#top_c0, top_r = np.load('yac_data/top_words/top_c0_full.npy'), np.load('yac_data/top_words/top_r_full.npy')

#t_top_c2, t_top_c1 = np.load('yac_data/top_words/t_top_c2_full.npy'), np.load('yac_data/top_words/t_top_c1_full.npy')
#t_top_c0, t_top_r = np.load('yac_data/top_words/t_top_c0_full.npy'), np.load('yac_data/top_words/t_top_r_full.npy')
#t_top = np.concatenate([t_top_c2, t_top_c1, t_top_c0, t_top_r], axis=1)
top_c2, top_c1, top_c0, top_r = get_important_words(Xtrain, vectorizer, w2v)
top = np.concatenate([top_c2, top_c1, top_c0, top_r], axis=1)

Getting data...


HBox(children=(IntProgress(value=0, max=97533), HTML(value='')))


...Done
Progress: 0%
Progress: 25%
Progress: 50%
Progress: 75%
Progress: 100%


## Getting grammes

In [55]:
#grammes_keys = get_grammems_keys(Xtrain)
#grammes_keys = np.load('grammes_keys.npy')
#grammes_keys = list(grammes_keys)

In [56]:
def get_grammems_sentence(sent, morph):
    if len(sent) == 0:
        return {}
    grammems = {}
    for word in sent:
        tags = list(morph.parse(word)[0].tag.grammemes)
        for tag in tags:
            if tag in grammems:
                grammems[tag] += 1 / len(sent)
            else:
                grammems[tag] = 1 / len(sent)
    return grammems

In [57]:
#gram_c2, gram_c1, gram_c0, gram_r = get_grammems(Xtrain, list(grammes_keys))
gram_c2, gram_c1 = np.load('yac_data/grammems/gram_c2.npy'), np.load('yac_data/grammems/gram_c1.npy')
gram_c0, gram_r = np.load('yac_data/grammems/gram_c0.npy'), np.load('yac_data/grammems/gram_r.npy')

In [58]:
#t_gram_c2, t_gram_c1, t_gram_c0, t_gram_r = get_grammems(Xtest, list(grammes_keys))
t_gram_c2, t_gram_c1 = np.load('yac_data/grammems/t_gram_c2.npy'), np.load('yac_data/grammems/t_gram_c1.npy')
t_gram_c0, t_gram_r = np.load('yac_data/grammems/t_gram_c0.npy'), np.load('yac_data/grammems/t_gram_r.npy')

## Getting META

In [59]:
def get_idf(X, vectorizer):
    data = []
    print('Getting data...')
    vocab = vectorizer.vocabulary_
    idf = vectorizer.idf_
    for sample in tqdm_notebook(X.values):
        idf_c2 = get_idf_sentence(sample[1].split(), vocab, idf)
        cw_c2 = len(sample[1])
        c2 = np.array([idf_c2, cw_c2])
            
        idf_c1 = get_idf_sentence(sample[2].split(), vocab, idf)
        cw_c1 = len(sample[2])
        c1 = np.array([idf_c1, cw_c1])
            
        idf_c0 = get_idf_sentence(sample[3].split(), vocab, idf)
        cw_c0 = len(sample[3])
        c0 = np.array([idf_c0, cw_c0])
            
        idf_r = get_idf_sentence(sample[5].split(), vocab, idf)
        cw_r = len(sample[5])
        r = np.array([idf_r, cw_r])
        
        data.append((c2, c1, c0, r))
        
    print('...Done')
    data = pd.DataFrame(data, columns=['c2', 'c1', 'c0', 'r'])

    c2 = np.zeros((len(data), 2))
    print('Progress: 0%')
    for i in range(len(data)):
        c2[i] = data.c2[i]
    c1 = np.zeros((len(data), 2))
    print('Progress: 25%')
    for i in range(len(data)):
        c1[i] = data.c1[i]
    c0 = np.zeros((len(data), 2))
    print('Progress: 50%')
    for i in range(len(data)):
        c0[i] = data.c0[i]
    r = np.zeros((len(data), 2))
    print('Progress: 75%')
    for i in range(len(data)):
        r[i] = data.r[i]
    print('Progress: 100%')
    return c2,c1,c0,r

In [60]:
meta_c2, meta_c1, meta_c0, meta_r = get_idf(Xtrain, vectorizer)
meta_x = np.concatenate([meta_c2, meta_c1, meta_c0, meta_r], axis=1)
meta_x.shape

Getting data...


HBox(children=(IntProgress(value=0, max=97533), HTML(value='')))


...Done
Progress: 0%
Progress: 25%
Progress: 50%
Progress: 75%
Progress: 100%


(97533, 8)

In [61]:
meta_c2_s = meta_c2.reshape(meta_c2.shape[0], 1, 2)
meta_c1_s = meta_c1.reshape(meta_c1.shape[0], 1, 2)
meta_c0_s = meta_c0.reshape(meta_c0.shape[0], 1, 2)
meta_r_s = meta_r.reshape(meta_r.shape[0], 1, 2)

meta_x_s = np.concatenate([meta_c2_s, meta_c1_s, meta_c0_s, meta_r_s], axis=1)

In [62]:
meta_x_s.shape

(97533, 4, 2)

In [63]:
t_meta_c2, t_meta_c1, t_meta_c0, t_meta_r = get_idf(Xtest, vectorizer)
t_meta_x = np.concatenate([t_meta_c2, t_meta_c1, t_meta_c0, t_meta_r], axis=1)
t_meta_x.shape

Getting data...


HBox(children=(IntProgress(value=0, max=9968), HTML(value='')))


...Done
Progress: 0%
Progress: 25%
Progress: 50%
Progress: 75%
Progress: 100%


(9968, 8)

In [64]:
t_meta_c2_s = t_meta_c2.reshape(t_meta_c2.shape[0], 1, 2)
t_meta_c1_s = t_meta_c1.reshape(t_meta_c1.shape[0], 1, 2)
t_meta_c0_s = t_meta_c0.reshape(t_meta_c0.shape[0], 1, 2)
t_meta_r_s = t_meta_r.reshape(t_meta_r.shape[0], 1, 2)

t_meta_x_s = np.concatenate([t_meta_c2_s, t_meta_c1_s, t_meta_c0_s, t_meta_r_s], axis=1)

## Getting w2v sums

In [65]:
def get_w2v_data(X, vectorizer=None, dim=100, do_idf=False):
    data = []
    print('Getting data...')
    if do_idf == True:
        vocab = vectorizer.vocabulary_
        idf = vectorizer.idf_
        dim += 2
    for sample in tqdm_notebook(X.values):
        c2 = w2v.get_sentence_vector(sample[1])
        c1 = w2v.get_sentence_vector(sample[2])
        c0 = w2v.get_sentence_vector(sample[3])
        r = w2v.get_sentence_vector(sample[5])
        
        if do_idf == True:
            idf_c2 = get_idf_sentence(sample[1].split(), vocab, idf)
            cw_c2 = len(sample[1])
            c2 = np.concatenate([c2, [idf_c2, cw_c2]])
            
            idf_c1 = get_idf_sentence(sample[2].split(), vocab, idf)
            cw_c1 = len(sample[2])
            c1 = np.concatenate([c1, [idf_c1, cw_c1]])
            
            idf_c0 = get_idf_sentence(sample[3].split(), vocab, idf)
            cw_c0 = len(sample[3])
            c0 = np.concatenate([c0, [idf_c0, cw_c0]])
            
            idf_r = get_idf_sentence(sample[5].split(), vocab, idf)
            cw_r = len(sample[5])
            r = np.concatenate([r, [idf_r, cw_r]])
        
        data.append((c2, c1, c0, r))
        
    print('...Done')
    data = pd.DataFrame(data, columns=['c2', 'c1', 'c0', 'r'])

    c2 = np.zeros((len(data), dim))
    print('Progress: 0%')
    for i in range(len(data)):
        c2[i] = data.c2[i]
    c1 = np.zeros((len(data), dim))
    print('Progress: 25%')
    for i in range(len(data)):
        c1[i] = data.c1[i]
    c0 = np.zeros((len(data), dim))
    print('Progress: 50%')
    for i in range(len(data)):
        c0[i] = data.c0[i]
    r = np.zeros((len(data), dim))
    print('Progress: 75%')
    for i in range(len(data)):
        r[i] = data.r[i]
    print('Progress: 100%')
    return c2,c1,c0,r

def get_intersection(X, vectorizer=None, dim=100, do_syn=False, do_idf=False):
    data = []
    print('Getting data...')
    if do_idf == True:
        vocab = vectorizer.vocabulary_
        idf = vectorizer.idf_
        dim += 2
    syn_vocab = {}
    for sample in tqdm_notebook(X.values):
        c2 = set(sample[1].split())
        c1 = set(sample[2].split())
        c0 = set(sample[3].split())
        if do_syn == False:
            r = set(sample[5].split())
        else:
            r = sample[5].split()
            r_extended = []
            for w_r in r:
                if w_r in syn_vocab:
                    synonyms = syn_vocab[w_r]
                else:
                    v_r = w2v.get_word_vector(w_r)
                    synonyms = m.similar_by_vector(v_r, topn=4)
                    syn_vocab[w_r] = synonyms
                #print(synonyms)
                synonyms = pd.DataFrame(synonyms, columns=['word', 'prob'])
                synonyms = synonyms.word.tolist()
                r_extended += synonyms
            r = set(r_extended)
        
        list_c2 = list(c2 & r)
        list_c1 = list(c1 & r)
        list_c0 = list(c0 & r)
        inter_c2 = w2v.get_sentence_vector(' '.join(list_c2))
        inter_c1 = w2v.get_sentence_vector(' '.join(list_c1))
        inter_c0 = w2v.get_sentence_vector(' '.join(list_c0))
        
        if do_idf == True:
            idf_c2 = get_idf_sentence(list_c2, vocab, idf)
            cw_c2 = len(list_c2)
            inter_c2 = np.concatenate([inter_c2, [idf_c2, cw_c2]])
            
            idf_c1 = get_idf_sentence(list_c1, vocab, idf)
            cw_c1 = len(list_c1)
            inter_c1 = np.concatenate([inter_c1, [idf_c1, cw_c1]])
            
            idf_c0 = get_idf_sentence(list_c0, vocab, idf)
            cw_c0 = len(list_c0)
            inter_c0 = np.concatenate([inter_c0, [idf_c0, cw_c0]])
        
        data.append((inter_c2, inter_c1, inter_c0))
    print('...Done')
    data = pd.DataFrame(data, columns=['c2', 'c1', 'c0'])

    c2 = np.zeros((len(data), dim))
    print('Progress: 0%')
    for i in range(len(data)):
        c2[i] = data.c2[i]
    c1 = np.zeros((len(data), dim))
    print('Progress: 30%')
    for i in range(len(data)):
        c1[i] = data.c1[i]
    c0 = np.zeros((len(data), dim))
    print('Progress: 65%')
    for i in range(len(data)):
        c0[i] = data.c0[i]
    print('Progress: 100%')
    return c2, c1, c0

def get_intersection_with_context(X, dim=100, do_idf=False):
    data = []
    print('Getting data...')
    if do_idf == True:
        vocab = vectorizer.vocabulary_
        idf = vectorizer.idf_
        dim += 2
    for sample in tqdm_notebook(X.values):
        c2 = set(sample[1].split())
        c1 = set(sample[2].split())
        c0 = set(sample[3].split())
        i1 = list(c2 & c1)
        i2 = list(c1 & c0)
        i3 = list(c0 & c2)
        
        inter_c2_c1 = w2v.get_sentence_vector(' '.join(i1))
        inter_c1_c0 = w2v.get_sentence_vector(' '.join(i2))
        inter_c0_c2 = w2v.get_sentence_vector(' '.join(i3))
        if do_idf == True:
            idf_c2 = get_idf_sentence(i1, vocab, idf)
            cw_c2 = len(i1)
            inter_c2_c1 = np.concatenate([inter_c2_c1, [idf_c2, cw_c2]])
            
            idf_c1 = get_idf_sentence(i2, vocab, idf)
            cw_c1 = len(i2)
            inter_c1_c0 = np.concatenate([inter_c1_c0, [idf_c1, cw_c1]])
            
            idf_c0 = get_idf_sentence(i3, vocab, idf)
            cw_c0 = len(i3)
            inter_c0_c2 = np.concatenate([inter_c0_c2, [idf_c0, cw_c0]])
        
        data.append((inter_c2_c1, inter_c1_c0, inter_c0_c2))
    print('...Done')
    data = pd.DataFrame(data, columns=['c2_c1', 'c1_c0', 'c0_c2'])

    c2_c1 = np.zeros((len(data), dim))
    print('Progress: 0%')
    for i in range(len(data)):
        c2_c1[i] = data.c2_c1[i]
    c1_c0 = np.zeros((len(data), dim))
    print('Progress: 30%')
    for i in range(len(data)):
        c1_c0[i] = data.c1_c0[i]
    c0_c2 = np.zeros((len(data), dim))
    print('Progress: 65%')
    for i in range(len(data)):
        c0_c2[i] = data.c0_c2[i]
    print('Progress: 100%')
    return c2_c1, c1_c0, c0_c2

def get_grammems_keys(X):
    keys = set()
    morph = pymorphy2.MorphAnalyzer()
    print('Getting data...')
    for sample in tqdm_notebook(X.values):
        grammems_c2 = get_grammems_sentence(sample[1].split(), morph)
        grammems_c1 = get_grammems_sentence(sample[2].split(), morph)
        grammems_c0 = get_grammems_sentence(sample[3].split(), morph)
        grammems_r = get_grammems_sentence(sample[5].split(), morph)
        
        keys.update((list(grammems_c2.keys())))
        keys.update(list(grammems_c1.keys()))
        keys.update((list(grammems_c0.keys())))
        keys.update((list(grammems_r.keys())))
            
    print('...Done')
    return keys

In [66]:
%%time
c2, c1, c0, r = get_w2v_data(Xtrain, vectorizer, 200)
#c2, c1 = np.load('yac_data/sums/c2.npy'), np.load('yac_data/sums/c1.npy')
#c0, r = np.load('yac_data/sums/c0.npy'), np.load('yac_data/sums/r.npy')

#t_c2, t_c1 = np.load('yac_data/sums/t_c2.npy'), np.load('yac_data/sums/t_c1.npy')
#t_c0, t_r = np.load('yac_data/sums/t_c0.npy'), np.load('yac_data/sums/t_r.npy')

Getting data...


HBox(children=(IntProgress(value=0, max=97533), HTML(value='')))


...Done
Progress: 0%
Progress: 25%
Progress: 50%
Progress: 75%
Progress: 100%
CPU times: user 22.2 s, sys: 6.74 s, total: 28.9 s
Wall time: 25.7 s


## Searching intersections with reply

In [67]:
%%time
inter_c2, inter_c1, inter_c0 = get_intersection(Xtrain, vectorizer, 200, False, False)
t_inter_c2, t_inter_c1, t_inter_c0 = get_intersection(Xtest, vectorizer, 200, False, False)
#inter_c2, inter_c1 = np.load('yac_data/inter/inter_syn_c2.npy'), np.load('yac_data/inter/inter_syn_c1.npy')
#inter_c0 = np.load('yac_data/inter/inter_syn_c0.npy')
#t_inter_c2, t_inter_c1 = np.load('yac_data/inter/t_inter_syn_c2.npy'), np.load('yac_data/inter/t_inter_syn_c1.npy')
#t_inter_c0 = np.load('yac_data/inter/t_inter_syn_c0.npy')

Getting data...


HBox(children=(IntProgress(value=0, max=97533), HTML(value='')))


...Done
Progress: 0%
Progress: 30%
Progress: 65%
Progress: 100%
Getting data...


HBox(children=(IntProgress(value=0, max=9968), HTML(value='')))


...Done
Progress: 0%
Progress: 30%
Progress: 65%
Progress: 100%
CPU times: user 12.9 s, sys: 3.04 s, total: 16 s
Wall time: 14.4 s


## Validation

In [68]:
np.random.seed(0)
validation_contexts = set(np.random.choice(np.unique(Xtrain.context_id), 3500, replace=False))
validation_mask = Xtrain.context_id.isin(validation_contexts)

In [69]:
validation = Xtrain[validation_mask].reset_index(drop=True)
train = Xtrain[~validation_mask].reset_index(drop=True)

In [70]:
c2_res = np.concatenate([c2, inter_c2], axis=1)
c1_res = np.concatenate([c1, inter_c1], axis=1)
c0_res = np.concatenate([c0, inter_c0], axis=1)
r_res = np.concatenate([r], axis=1)
#intr_cont = np.concatenate([c2_c1, c1_c0, c0_c2], axis=1)

c2_res.shape

#t_c2_res = np.concatenate([t_c2, t_inter_c2], axis=1)
#t_c1_res = np.concatenate([t_c1, t_inter_c1], axis=1)
#t_c0_res = np.concatenate([t_c0, t_inter_c0], axis=1)
#t_r_res = np.concatenate([t_r], axis=1)

test_top = top[validation_mask]
train_top = top[~validation_mask]

In [71]:
train_c2_res = c2_res[~validation_mask]
train_c1_res = c1_res[~validation_mask]
train_c0_res = c0_res[~validation_mask]
train_r_res = r_res[~validation_mask]
train_meta_x = meta_x[~validation_mask]
train_top_c2 = top_c2[~validation_mask]
train_top_c1 = top_c1[~validation_mask]
train_top_c0 = top_c0[~validation_mask]
train_top_r = top_r[~validation_mask]

In [72]:
test_c2_res = c2_res[validation_mask]
test_c1_res = c1_res[validation_mask]
test_c0_res = c0_res[validation_mask]
test_r_res = r_res[validation_mask]
test_meta_x = meta_x[validation_mask]
test_top_c2 = top_c2[validation_mask]
test_top_c1 = top_c1[validation_mask]
test_top_c0 = top_c0[validation_mask]
test_top_r = top_r[validation_mask]

In [73]:
test_meta_x_s = meta_x_s[validation_mask]
train_meta_x_s = meta_x_s[~validation_mask]

## Making NN

In [74]:
from keras.layers import Conv2D, MaxPool1D, Permute, Add
from keras import backend as K 

In [86]:
def get_top_model_with_meta_cnn(dim_c2=100, dim_c1=100, dim_c0=100, dim_r=100, meta_dim=8):
    input_c2 = Input(shape=(dim_c2,))
    input_c1 = Input(shape=(dim_c1,))
    input_c0 = Input(shape=(dim_c0,))
    input_r = Input(shape=(dim_r,))
    input_meta = Input(shape=(meta_dim,))

    dl_c2 = Dense(512, input_dim=1, activation='relu')(input_c2)
    dl_c2 = Dropout(0.5)(dl_c2)
    dl_c2 = Dense(1024, activation='relu')(dl_c2)
    dl_c2 = BatchNormalization()(dl_c2)
    
    dl_c1 = Dense(512,input_dim=1, activation='relu')(input_c1)
    dl_c1 = Dropout(0.5)(dl_c1)
    dl_c1 = Dense(1024, activation='relu')(dl_c1)
    dl_c1 = BatchNormalization()(dl_c1)
    
    dl_c0 = Dense(512, input_dim=1, activation='relu')(input_c0)
    dl_c0 = Dropout(0.5)(dl_c0)
    dl_c0 = Dense(1024, activation='relu')(dl_c0)
    dl_c0 = BatchNormalization()(dl_c0)
    
    dl_r = Dense(512,input_dim=1, activation='relu')(input_r)
    dl_r = Dropout(0.5)(dl_r)
    dl_r = Dense(1024, activation='relu')(dl_r)
    dl_r = BatchNormalization()(dl_r)
    
    dl_meta = Dense(256,input_dim=1, activation='relu')(input_meta)
    dl_meta = Dropout(0.5)(dl_meta)
    dl_meta = Dense(512, activation='relu')(dl_meta)
    dl_meta = BatchNormalization()(dl_meta)
    
    out1 = Concatenate()([dl_c2, dl_c1, dl_c0, dl_r, dl_meta])
    
    x1 = Dense(2048, activation='relu')(out1)
    x1 = Dropout(0.5)(x1)
    x1 = Dense(2048, activation='relu')(x1)
    x1 = BatchNormalization()(x1)
    
    input_top = Input(shape=(20, 200, ))
    
    cl_t0 = Conv1D(200, 1, activation='relu')(input_top)
    cl_t0 = MaxPooling1D(9)(cl_t0)
    cl_t0 = Flatten()(cl_t0)
    cl_t0 = Dropout(0.5)(cl_t0)
    
    cl_t1 = Conv1D(200, 2, activation='relu')(input_top)
    cl_t1 = MaxPooling1D(6)(cl_t1)
    cl_t1 = Flatten()(cl_t1)
    cl_t1 = Dropout(0.5)(cl_t1)
    
    cl_t2 = Conv1D(200, 3, activation='relu')(input_top)
    cl_t2 = MaxPooling1D(6)(cl_t2)
    cl_t2 = Flatten()(cl_t2)
    cl_t2 = Dropout(0.5)(cl_t2)
    
    cl_t3 = Conv1D(200, 4, activation='relu')(input_top)
    cl_t3 = MaxPooling1D(6)(cl_t3)
    cl_t3 = Flatten()(cl_t3)
    cl_t3 = Dropout(0.5)(cl_t3)
    
    cl_t4 = Conv1D(200, 5, activation='relu')(input_top)
    cl_t4 = MaxPooling1D(6)(cl_t4)
    cl_t4 = Flatten()(cl_t4)
    cl_t4 = Dropout(0.5)(cl_t4)
    
    cl_t5 = Conv1D(200, 6, activation='relu')(input_top)
    cl_t5 = MaxPooling1D(6)(cl_t5)
    cl_t5 = Flatten()(cl_t5)
    cl_t5 = Dropout(0.5)(cl_t5)
    
    cl_t6 = Conv1D(200, 7, activation='relu')(input_top)
    cl_t6 = MaxPooling1D(6)(cl_t6)
    cl_t6 = Flatten()(cl_t6)
    cl_t6 = Dropout(0.5)(cl_t6)
    
    out2 = Concatenate()([cl_t0, cl_t1, cl_t2, cl_t3, cl_t4, cl_t5])

    x2 = Dense(2048, activation='relu')(out2)
    x2 = Dropout(0.5)(x2)
    x2 = Dense(2048, activation='relu')(x2)
    x2 = BatchNormalization()(x2)
    #x2 = Dense(3, activation='softmax')(x2)
    
    out = Concatenate()([x1, x2])
  
    x = Dense(2048, activation='relu')(out)
    x = Dropout(0.4)(x)
    x = Dense(4096, activation='relu')(x)
    x = BatchNormalization()(x)
    x = Dense(3, activation='softmax')(x)

    model = Model(input=[input_c2, input_c1, input_c0, input_r, input_meta, 
                         input_top], output=x)

    model.compile(loss='categorical_crossentropy',
                  optimizer='adam',
                  metrics=['acc'])
    model.summary()
    return model

In [97]:
model = get_top_model_with_meta_cnn_old(c2_res.shape[1],
                         c2_res.shape[1],
                         c2_res.shape[1],
                         r_res.shape[1],
                         8)

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_19 (InputLayer)           (None, 400)          0                                            
__________________________________________________________________________________________________
input_20 (InputLayer)           (None, 400)          0                                            
__________________________________________________________________________________________________
input_21 (InputLayer)           (None, 400)          0                                            
__________________________________________________________________________________________________
input_22 (InputLayer)           (None, 200)          0                                            
__________________________________________________________________________________________________
input_23 (

In [111]:
def get_top_model_with_meta_cnn_old(dim_c2=100, dim_c1=100, dim_c0=100, dim_r=100, meta_dim=8):
    input_c2 = Input(shape=(dim_c2,))
    input_c1 = Input(shape=(dim_c1,))
    input_c0 = Input(shape=(dim_c0,))
    input_r = Input(shape=(dim_r,))
    input_meta = Input(shape=(meta_dim,))

    dl_c2 = Dense(512, input_dim=1, activation='relu')(input_c2)
    dl_c2 = Dropout(0.5)(dl_c2)
    dl_c2 = Dense(1024, activation='relu')(dl_c2)
    dl_c2 = BatchNormalization()(dl_c2)
    
    dl_c1 = Dense(512,input_dim=1, activation='relu')(input_c1)
    dl_c1 = Dropout(0.5)(dl_c1)
    dl_c1 = Dense(1024, activation='relu')(dl_c1)
    dl_c1 = BatchNormalization()(dl_c1)
    
    dl_c0 = Dense(512, input_dim=1, activation='relu')(input_c0)
    dl_c0 = Dropout(0.5)(dl_c0)
    dl_c0 = Dense(1024, activation='relu')(dl_c0)
    dl_c0 = BatchNormalization()(dl_c0)
    
    dl_r = Dense(512,input_dim=1, activation='relu')(input_r)
    dl_r = Dropout(0.5)(dl_r)
    dl_r = Dense(1024, activation='relu')(dl_r)
    dl_r = BatchNormalization()(dl_r)
    
    dl_meta = Dense(256,input_dim=1, activation='relu')(input_meta)
    dl_meta = Dropout(0.5)(dl_meta)
    dl_meta = Dense(512, activation='relu')(dl_meta)
    dl_meta = BatchNormalization()(dl_meta)
    
    out1 = Concatenate()([dl_c2, dl_c1, dl_c0, dl_r, dl_meta])
    
    x1 = Dense(2048, activation='relu')(out1)
    x1 = Dropout(0.5)(x1)
    x1 = Dense(2048, activation='relu')(x1)
    x1 = BatchNormalization()(x1)
    
    input_top = Input(shape=(20, 200, ))
    
    cl_t0 = Conv1D(200, 1, activation='relu')(input_top)
    cl_t0 = MaxPooling1D(10)(cl_t0)
    cl_t0 = Flatten()(cl_t0)
    cl_t0 = Dropout(0.5)(cl_t0)
    
    cl_t1 = Conv1D(200, 2, activation='relu')(input_top)
    cl_t1 = MaxPooling1D(10)(cl_t1)
    cl_t1 = Flatten()(cl_t1)
    cl_t1 = Dropout(0.5)(cl_t1)
    
    cl_t2 = Conv1D(200, 3, activation='relu')(input_top)
    cl_t2 = MaxPooling1D(10)(cl_t2)
    cl_t2 = Flatten()(cl_t2)
    cl_t2 = Dropout(0.5)(cl_t2)
    
    cl_t3 = Conv1D(200, 4, activation='relu')(input_top)
    cl_t3 = MaxPooling1D(10)(cl_t3)
    cl_t3 = Flatten()(cl_t3)
    cl_t3 = Dropout(0.5)(cl_t3)
    
    cl_t4 = Conv1D(200, 5, activation='relu')(input_top)
    cl_t4 = MaxPooling1D(10)(cl_t4)
    cl_t4 = Flatten()(cl_t4)
    cl_t4 = Dropout(0.5)(cl_t4)
    
    cl_t5 = Conv1D(200, 6, activation='relu')(input_top)
    cl_t5 = MaxPooling1D(10)(cl_t5)
    cl_t5 = Flatten()(cl_t5)
    cl_t5 = Dropout(0.5)(cl_t5)
    
    cl_t6 = Conv1D(200, 7, activation='relu')(input_top)
    cl_t6 = MaxPooling1D(10)(cl_t6)
    cl_t6 = Flatten()(cl_t6)
    cl_t6 = Dropout(0.5)(cl_t6)
    
    out2 = Concatenate()([cl_t0, cl_t1, cl_t2, cl_t3, cl_t4, cl_t5, cl_t6])

    x2 = Dense(2048, activation='relu')(out2)
    x2 = Dropout(0.5)(x2)
    x2 = Dense(2048, activation='relu')(x2)
    x2 = BatchNormalization()(x2)
    #x2 = Dense(3, activation='softmax')(x2)
    
    out = Concatenate()([x1, x2])
  
    x = Dense(4096, activation='relu')(out)
    x = Dropout(0.4)(x)
    x = Dense(4096, activation='relu')(x)
    x = BatchNormalization()(x)
    x = Dense(3, activation='softmax')(x)

    model = Model(input=[input_c2, input_c1, input_c0, input_r, input_meta, 
                         input_top], output=x)

    model.compile(loss='categorical_crossentropy',
                  optimizer='adam',
                  metrics=['acc'])
    model.summary()
    return model

## Callbacks engineering 

In [112]:
checkpoint = ModelCheckpoint(
    'yac_weights/np-task1-yac-weights_model_eranewOLDBIG6.{epoch:02d}-{val_acc:.2f}.hdf5', 
    monitor='val_acc', verbose=0, save_best_only=False, save_weights_only=False, mode='auto', period=1)

learning_rate_reduction = ReduceLROnPlateau(monitor='val_acc', 
                                            patience=1, 
                                            verbose=1, 
                                            factor=0.2, 
                                            min_lr=0.00001)

earlyStopping = EarlyStopping(monitor='val_loss',
                                              min_delta=0, 
                                              patience=5, 
                                              verbose=1, 
                                              mode='auto')

In [113]:
np.random.seed(42)
model.fit([train_c2_res, train_c1_res, train_c0_res, train_r_res, train_meta_x,train_top],
          label[~validation_mask],
          validation_data = ([test_c2_res, test_c1_res, test_c0_res, test_r_res, test_meta_x, test_top], label[validation_mask]),
          nb_epoch=30, sample_weight=Xtrain.confidence.values[~validation_mask],
          batch_size=64, 
          callbacks=[earlyStopping, learning_rate_reduction, checkpoint],
          )

Train on 77591 samples, validate on 19942 samples
Epoch 1/30
77591/77591 [==============================] - 110s 1ms/step - loss: 0.5809 - acc: 0.6859 - val_loss: 0.8163 - val_acc: 0.6064
Epoch 2/30
77591/77591 [==============================] - 110s 1ms/step - loss: 0.5735 - acc: 0.6900 - val_loss: 0.8206 - val_acc: 0.6065
Epoch 3/30
77591/77591 [==============================] - 110s 1ms/step - loss: 0.5676 - acc: 0.6937 - val_loss: 0.8223 - val_acc: 0.6049
Epoch 4/30
77591/77591 [==============================] - 110s 1ms/step - loss: 0.5642 - acc: 0.6961 - val_loss: 0.8223 - val_acc: 0.6059
Epoch 5/30
77591/77591 [==============================] - 110s 1ms/step - loss: 0.5583 - acc: 0.6986 - val_loss: 0.8226 - val_acc: 0.6063
Epoch 6/30
77591/77591 [==============================] - 110s 1ms/step - loss: 0.5560 - acc: 0.7010 - val_loss: 0.8263 - val_acc: 0.6069
Epoch 00006: early stopping


In [326]:
p = model.predict([t_c2_res,t_c1_res,t_c0_res, t_r_res, t_meta_x, t_top], verbose=1)

9968/9968 [==============================] - 4s 355us/step


In [328]:
sub.to_csv('subs/sub_NEW44.csv', index=False, sep=' ')

In [129]:
model.load_weights('yac_weights/np-task1-yac-weights_model_era.06-0.60.hdf5')

In [123]:
model.save_weights('yac_weights/88077_val.hdf5')

In [126]:
model.load_weights('yac_weights/np-task1-yac-weights_model_eranewOLDBIG6.03-0.60.hdf5')

In [ ]:
p = model.predict([test_c2_res, test_c1_res, test_c0_res, test_r_res, test_meta_x, test_top], verbose=1)

validation['scores'] = p.dot([-1, 0, 1])
sub = validation.groupby('context_id').apply(
    lambda x: x.sort_values('scores', ascending=False).reply_id
).reset_index(level=0)

evaluate(sub, validation)

19942/19942 [==============================] - 6s 301us/step


In [36]:
def nDCG(group):
    denom = np.log2(2 + np.arange(len(group)))
    IDCG = np.sum(group.sort_values('label', ascending=False).label.values / denom)
    if IDCG == 0:
        return 1
    else:
        DCG = np.sum(group.label.values / denom)
        return DCG / IDCG


def evaluate(sub, truth):
    return sub.join(
        truth.set_index(['context_id', 'reply_id']).label,
        how='left', on=['context_id', 'reply_id']
    ).groupby('context_id').apply(nDCG).mean() * 100000

In [276]:
model.load_weights('yac_weights/87982_val.hdf5')

In [63]:
np.save('prob2.npy', p)

In [232]:
model_new = get_top_model_with_meta_cnn_vanila_gru(c2_res.shape[1],
                         c2_res.shape[1],
                         c2_res.shape[1],
                         r_res.shape[1],
                         8)

In [233]:
checkpoint = ModelCheckpoint(
    'yac_weights/np-task1-yac-weights_model_eranewNEWNEW3.{epoch:02d}-{val_acc:.2f}.hdf5', 
    monitor='val_acc', verbose=0, save_best_only=False, save_weights_only=False, mode='auto', period=1)

learning_rate_reduction = ReduceLROnPlateau(monitor='val_acc', 
                                            patience=1, 
                                            verbose=1, 
                                            factor=0.3, 
                                            min_lr=0.00001)

earlyStopping = EarlyStopping(monitor='val_loss',
                                              min_delta=0, 
                                              patience=5, 
                                              verbose=1, 
                                              mode='auto')

In [234]:
model_new.compile(loss='categorical_crossentropy',
                  optimizer='adam',
                  metrics=['acc'])
model_new.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_183 (InputLayer)          (None, 400)          0                                            
__________________________________________________________________________________________________
input_184 (InputLayer)          (None, 400)          0                                            
__________________________________________________________________________________________________
input_185 (InputLayer)          (None, 400)          0                                            
__________________________________________________________________________________________________
input_186 (InputLayer)          (None, 200)          0                                            
__________________________________________________________________________________________________
input_187 

In [236]:
np.random.seed(42)
np.random.RandomState(42)
model_new.fit([train_c2_res, train_c1_res, train_c0_res, train_r_res, train_meta_x, train_top, 
           train_top_c0, train_top_r],
          label[~validation_mask],
          validation_data = ([test_c2_res, test_c1_res, test_c0_res, test_r_res, test_meta_x,
                              test_top, test_top_c0, test_top_r], label[validation_mask]),
          nb_epoch=30, sample_weight=Xtrain.confidence.values[~validation_mask],
          batch_size=78, 
          callbacks=[earlyStopping, learning_rate_reduction, checkpoint],
          )

Train on 77591 samples, validate on 19942 samples
Epoch 1/30
77591/77591 [==============================] - 39s 498us/step - loss: 0.7380 - acc: 0.5691 - val_loss: 0.8829 - val_acc: 0.5795
Epoch 2/30
77591/77591 [==============================] - 38s 486us/step - loss: 0.7144 - acc: 0.5918 - val_loss: 0.8633 - val_acc: 0.5609
Epoch 3/30
77591/77591 [==============================] - 38s 485us/step - loss: 0.7047 - acc: 0.6064 - val_loss: 0.8447 - val_acc: 0.5815
Epoch 4/30
77591/77591 [==============================] - 38s 485us/step - loss: 0.6838 - acc: 0.6228 - val_loss: 0.8163 - val_acc: 0.6032
Epoch 5/30
77591/77591 [==============================] - 38s 484us/step - loss: 0.6510 - acc: 0.6403 - val_loss: 0.8504 - val_acc: 0.5986
Epoch 6/30
77591/77591 [==============================] - 38s 487us/step - loss: 0.6053 - acc: 0.6664 - val_loss: 1.2073 - val_acc: 0.4160

Epoch 00006: ReduceLROnPlateau reducing learning rate to 0.0003000000142492354.
Epoch 7/30
77591/77591 [===========

In [248]:
model_new.load_weights('yac_weights/np-task1-yac-weights_model_eranewNEWNEW3.05-0.60.hdf5')

In [249]:
p = model_new.predict([test_c2_res, test_c1_res, test_c0_res, test_r_res, test_meta_x,
                   test_top,  test_top_c0, test_top_r], verbose=1)



19942/19942 [==============================] - 5s 227us/step


In [250]:
validation['scores'] = p.dot([-1, 0, 1])
sub = validation.groupby('context_id').apply(
    lambda x: x.sort_values('scores', ascending=False).reply_id
).reset_index(level=0)

evaluate(sub, validation)

87322.86650461525

In [127]:
model.save_weights('yac_weights/87607_val.hdf5')

In [56]:
model.load_weights('yac_weights/np-task1-yac-weights_model_eranew.06-0.60.hdf5')

In [81]:
p = model.predict([test_c2_res,test_c1_res,test_c0_res, test_r_res, test_meta_x,
                    test_top_c2, test_top_c1, test_top_c0, test_top_r], verbose=1)

validation['scores'] = p.dot([-1, 0, 1])
sub = validation.groupby('context_id').apply(
    lambda x: x.sort_values('scores', ascending=False).reply_id
).reset_index(level=0)

evaluate(sub, validation)

ValueError: Error when checking model : the list of Numpy arrays that you are passing to your model is not the size the model expected. Expected to see 6 array(s), but instead got the following list of 9 arrays: [array([[ 0.05075625,  0.02434491,  0.02742608, ...,  0.        ,
         0.        ,  0.        ],
       [ 0.05075625,  0.02434491,  0.02742608, ...,  0.        ,
         0.        ,  0.        ],...

## Making Submission

In [155]:
pred = model.predict([test_c2_res,test_c1_res,test_c0_res, test_r_res, test_meta_x,
                    test_top_c2, test_top_c1, test_top_c0, test_top_r], verbose=1)

19942/19942 [==============================] - 10s 503us/step


In [91]:
sub.to_csv('subs/sub_new_OLD_META2.csv', index=False, sep=' ')

## Scoring

Old attempts

In [1]:
for index, filtersize in enumerate(filtersize_list):
    nb_filter = number_of_filters_per_filtersize[index]
    pool_length = pool_length_list[index]
    conv = Conv1D(nb_filter=nb_filter, filter_length=filtersize, activation='relu')(input_context)
    pool = MaxPooling1D(pool_length=pool_length)(conv)
    flatten = Flatten()(pool)
    flatten = Dropout(0.5)(flatten)
    conv_list_context.append(flatten)
    
for index, filtersize in enumerate(filtersize_list):
    nb_filter = number_of_filters_per_filtersize[index]
    pool_length = pool_length_list[index]
    conv = Conv1D(nb_filter=nb_filter, filter_length=filtersize, activation='relu')(input_reply)
    pool = MaxPooling1D(pool_length=pool_length)(conv)
    flatten = Flatten()(pool)
    flatten = Dropout(0.5)(flatten)
    conv_list_reply.append(flatten)

out_context = Merge(mode='concat')(conv_list_context)
out_reply = Merge(mode='concat')(conv_list_reply)

out = Concatenate()([out_context, out_reply])

x = Dropout(0.5)(out)
x = Dense(100)(x)
x = Dropout(0.5)(x)
x = Activation('relu')(x)
x = Dense(3, activation='softmax')(x)

model = Model(input=[input_1, input_2], output=x)

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['acc'])
model.summary()

model.fit([train_c, train_r], label,
          validation_split=0.15,
          nb_epoch=5,
          batch_size=64, 
          )

NameError: name 't' is not defined

In [4]:
from keras.layers import Convolution1D,  Convolution2D, MaxPooling1D, Embedding, Reshape, LSTM, BatchNormalization
from keras.layers import Dense, Input, Flatten, Dropout, Merge, Activation, Concatenate
from keras.models import Model, Sequential
import numpy as np
import os
import tensorflow as tf
if tf.test.gpu_device_name():
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))
else:
    print("Please install GPU version of TF")

Default GPU Device: /device:GPU:0


In [7]:
folder = '../tf_dataset_features_npy/'

speakers_list = os.listdir(folder)
speakers_set = set(speakers_list)
test_speakers = np.random.choice(speakers_list, 300, replace=False)
train_speakers = np.array(list(speakers_set - set(test_speakers)))

def create_sample_from_audio(file, sample_length=11):
    words_list = os.listdir(file)
    word = np.random.choice(words_list, 1)[0]
    samples_list = os.listdir(os.path.join(file, word))
    sample = np.random.choice(samples_list, 1)[0]
    features = np.load(os.path.join(file, word, sample))
    len = features.shape[1]
    i = np.random.randint(sample_length, len)
    sample = features[:, i - sample_length : i]
    #print(sample.shape)
    return sample#.swapaxes(0, 1)

def batch_generator(folder, speakers_list, batch_size=32, sample_length=11):
    people_files = os.listdir(folder)
    while True:
        x_batch1, x_batch2, y_batch = [], [],[]
        for i in range(batch_size):
            if np.random.uniform(0,1) < 0.5: #from one man
                speaker = np.random.choice(speakers_list, 1)[0]
                sample_part1 = create_sample_from_audio(os.path.join(folder, speaker), sample_length)
                sample_part2 = create_sample_from_audio(os.path.join(folder, speaker), sample_length)
                y = 1
            else: #from two different people
                speaker1, speaker2 = np.random.choice(speakers_list, 2, replace=False)  
                sample_part1 = create_sample_from_audio(os.path.join(folder, speaker1), sample_length)
                sample_part2 = create_sample_from_audio(os.path.join(folder, speaker2), sample_length)
                y = 0
                
            x_batch1.append(sample_part1.reshape(11, 128))
            x_batch2.append(sample_part2.reshape(11, 128))
            y_batch.append(y)
            
        x_batch1 = np.array(x_batch1)
        x_batch2 = np.array(x_batch2)
        y_batch = np.array(y_batch)

        yield [x_batch1, x_batch2], y_batch

def generate_model(sample_length=11):
    inputs1 = Input(shape=(11, 128, ))
    inputs2 = Input(shape=(11, 128, ))
    
    conv_layer = Conv1D(nb_filter=30, filter_length=3,  activation = 'relu')
    conv1 = conv_layer(inputs1)
    conv2 = conv_layer(inputs2)
    
    pool1 = MaxPooling1D()(conv1)
    pool2 = MaxPooling1D()(conv2)
    pool1 = Flatten()(conv1)
    pool2 = Flatten()(pool2)
    merged = Concatenate()([pool1, pool2])
    
    classify = Dense(10, activation = 'relu')(merged)
    classify = Dense(1, activation = 'sigmoid')(classify)
    model = Model(inputs=[inputs1, inputs2], outputs=classify)
    model.summary()
    return model

SAMPLE_LENGTH = 11
model = generate_model(SAMPLE_LENGTH)
model.compile(loss='binary_crossentropy', optimizer='adam', metrics = ['accuracy'])

model.fit_generator(batch_generator(folder, train_speakers, 128, SAMPLE_LENGTH),
                   steps_per_epoch =1000,
                   nb_epoch = 100,
                   validation_data = batch_generator(folder, test_speakers, 2, SAMPLE_LENGTH),
                   validation_steps = 100)



Instructions for updating:
`NHWC` for data_format is deprecated, use `NWC` instead
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, 11, 128)      0                                            
__________________________________________________________________________________________________
input_4 (InputLayer)            (None, 11, 128)      0                                            
__________________________________________________________________________________________________
conv1d_1 (Conv1D)               (None, 9, 30)        11550       input_3[0][0]                    
                                                                 input_4[0][0]                    
__________________________________________________________________________________________________
max_pooling1d_2 (MaxPoolin

KeyboardInterrupt: 

In [13]:
inputs1 = Input(shape=(11, 128,))
conv1 = Conv1D(30, 3,  activation = 'relu')(inputs1)
conv1 = Flatten()(conv1)
classify = Dense(1, activation = 'sigmoid')(conv1)
model = Model(inputs=[inputs1], outputs=classify)

Instructions for updating:
`NHWC` for data_format is deprecated, use `NWC` instead


In [14]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics = ['accuracy'])

In [15]:
x = np.zeros((1, 11, 128))

In [16]:
y = np.zeros((1))

In [17]:
model.fit(x, y)

Epoch 1/1
1/1 [==============================] - 0s 177ms/step - loss: 0.6931 - acc: 1.0000
